In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [1]:
# 0.读取数据
data = pd.read_csv('./FBlocation/train.csv')
data.head()

NameError: name 'pd' is not defined

In [39]:
# 1.基本数据处理
# 1.1 缩小数据范围
partial_data = data.query('x>2.0 & y<2.5 & y >2.0 & y<2.5')

82          9313893105
91          4639582171
93          2677147592
101         3762957785
122         8663061061
               ...    
29117832    3491016295
29117892    8570161822
29117938    7946582171
29117956    3895820311
29118020    7028698129
Name: place_id, Length: 1149346, dtype: int64

In [43]:
# 2.2 选择时间特征
time = pd.to_datetime(partial_data['time'],unit = 's')
time = pd.DatetimeIndex(time)
partial_data['hour'] = time.hour
partial_data['day'] = time.day
partial_data['weekday'] = time.weekday
partial_data.head()

,row_id,x,y,accuracy,time,place_id,hour,day,weekday
82,82,2.9265,2.3590,55,415119,9313893105,19,5,0
91,91,3.5149,2.4510,3,202338,4639582171,8,3,5
93,93,4.8579,2.4629,33,643386,2677147592,10,8,3
101,101,3.4777,2.0938,71,115352,3762957785,8,2,4
122,122,4.7218,2.4048,60,338360,8663061061,21,4,6


In [44]:
# 2.3 按照所去的地方分类,小于三次的剔除掉
partial_count = partial_data.groupby('place_id').count()
partial_count = partial_count[partial_count.row_id > 3]
partial_data = partial_data[partial_data.place_id.isin(partial_count.index)]
partial_data.head()

,row_id,x,y,accuracy,time,place_id,hour,day,weekday
82,82,2.9265,2.3590,55,415119,9313893105,19,5,0
91,91,3.5149,2.4510,3,202338,4639582171,8,3,5
93,93,4.8579,2.4629,33,643386,2677147592,10,8,3
101,101,3.4777,2.0938,71,115352,3762957785,8,2,4
122,122,4.7218,2.4048,60,338360,8663061061,21,4,6


In [45]:
# 2.4 确定特征值和目标值
x = partial_data[['x','y','accuracy','hour','day','weekday']]
y = partial_data['place_id']
# 2.5 分割数据集
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size= 0.2,random_state= 20)

In [46]:
# 3 特征工程
# 3.1 数据标准化
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.fit_transform(x_test)

In [55]:
# 4 模型训练
# 4.1 交叉验证与网格搜索
estimator = KNeighborsClassifier()
# 定义超参数
param_grid = {'n_neighbors':[3,5,7,9]}
# 定义几折交叉验证，此处为4
estimator = GridSearchCV(estimator,param_grid = param_grid,cv = 4,n_jobs = -1)
estimator.fit(x_train,y_train)

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


GridSearchCV(cv=4, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [3, 5, 7, 9]})

In [56]:
# 5 预测结果
score = estimator.score(x_test,y_test)
print(score)
print(estimator.best_score_)
print(estimator.best_estimator_)

0.2226921165299129
0.20832602176748913
KNeighborsClassifier(n_neighbors=9)
